In [15]:
import os
import json
import requests

In [16]:
def getTables(html):
    splits = html.text.split('<table class="wikitable')
    if len(splits) != 2:
        print("there are the wrong number of tables on this page: " + str(len(splits) -1))
        return 
    htmlTable = splits[1].split('</table>')[0]
    if '<table' in htmlTable:
        print('there was a subtable we cannot deal with')
        return
    return htmlTable

In [17]:
def getRawHtml(table):
    pgId = table['pgId']
    url = 'https://en.wikipedia.org/?curid=' + str(pgId)
    response = requests.get(url)
    if getTables(response):
        print(pgId)

In [ ]:
input_file_path = '../data/100tables.json'
file = open(input_file_path, 'r')
for line in file:
    table = json.loads(line)
    getRawHtml(table)

there are the wrong number of tables on this page: 11
1000006
10000088
10000218
